## 3.2. 线性回归的从零开始实现¶
### 3.2.1 生成数据集
![image-Snipaste_2024-01-24_16-36-12.png](./assets/Snipaste_2024-01-24_16-36-12.png)


In [4]:

from mxnet import autograd, nd
import random

num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2

features = nd.random.normal(scale=1, shape=(num_examples, num_inputs))
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += nd.random.normal(scale=0.01, shape=labels.shape)


# 读取数据集函数
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        j = nd.array(indices[i:min(i + batch_size, num_examples)])
        yield features.take(j), labels.take(j)


batch_size = 10
for X, y in data_iter(batch_size, features, labels):
    print(X)
    print(y)
    break


[[ 0.590983    0.2707795 ]
 [ 0.7413724   0.2915591 ]
 [ 2.2544928  -1.9625562 ]
 [ 0.31813768  0.41370213]
 [ 2.0036721  -0.50719374]
 [ 1.3023337  -0.46279505]
 [-0.6202043  -1.1562326 ]
 [-0.14444932  2.0692244 ]
 [-0.9664803  -1.6631418 ]
 [ 1.7235638   0.37934572]]
<NDArray 10x2 @cpu(0)>

[ 4.466116   4.6982875 15.384071   3.455777   9.932828   8.392988
  6.90768   -3.1222196  7.9330177  6.3534727]
<NDArray 10 @cpu(0)>


In [5]:
# 初始化模型参数
# 我们将权重初始化成均值为0、标准差为0.01的正态随机数，偏差则初始化成0。

w = nd.random.normal(scale=0.01, shape=(num_inputs, 1))
b = nd.zeros(shape=(1,))

print(w)
print(b)


# 模型函数
def linreg(X, w, b):
    return nd.dot(X, w) + b


# 损失函数
def squared_loss(y_hat, y):
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2


# 优化函数
def sgd(params, lr, batch_size):
    for param in params:
        param[:] = param - lr * param.grad / batch_size



lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss

for epoch in range(num_epochs):  # 训练模型一共需要num_epochs个迭代周期
    # 在每一个迭代周期中，会使用训练数据集中所有样本一次（假设样本数能够被批量大小整除）。X
    # 和y分别是小批量样本的特征和标签
    for X, y in data_iter(batch_size, features, labels):
        with autograd.record():
            l = loss(net(X, w, b), y)  # l是有关小批量X和y的损失
        l.backward()  # 小批量的损失对模型参数求梯度
        sgd([w, b], lr, batch_size)  # 使用小批量随机梯度下降迭代模型参数
    train_l = loss(net(features, w, b), labels)
    print('epoch %d, loss %f' % (epoch + 1, train_l.mean().asnumpy()))


[[-0.01546639]
 [-0.00479909]]
<NDArray 2x1 @cpu(0)>

[0.]
<NDArray 1 @cpu(0)>
